In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml

# Arizona State University

In [ ]:
ASU_scholarships = pd.DataFrame(columns=['Name of Scholarship', 'Description', 'Awarding Body', 'Type of Scholarship', 'Category', 'Application Link',
                                         'Eligibility Criteria', 'Application Dedline', 'Award Amount'])

In [ ]:
ASU_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount,Application Info


In [ ]:
url = 'https://scholarships.asu.edu/'

In [ ]:
html = requests.get(url)

In [ ]:
html.status_code

200

In [ ]:
soup1 = BeautifulSoup(html.text, 'lxml')

In [ ]:
def sub_pages(base,lnk):
  html = requests.get(lnk)
  soup = BeautifulSoup(html.text, 'lxml')

  sm_text = soup.find('div', attrs={'class': 'field-type-text-with-summary'}).find('div', attrs={'class': 'even'}).findAll('p')
  summary = ''
  for p in sm_text:
    summary += p.text
  
  amount = soup.find('div', attrs={'class' : 'field-name-field-s-amount'}).find('div', attrs={'class': 'even'}).text

  criteria = {}
  for items in soup.find('fieldset', attrs={'class': 'group-criteria'}).findAll('div', attrs={'class' : 'field'}):
    key = items.find('div', attrs = {'class': 'field-label'}).text.strip()
    value = items.find('div', attrs={'class': 'even'}).text.strip()
    criteria[key] = value
  
  awarding_body = soup.find('fieldset', attrs={'class': 'group-contact'}).find('div', attrs={'class': 'even'}).text
  
  
  deadline = soup.find('div', attrs={'class': 'field-name-scholarship-deadline'}).find('div', attrs={'class': 'even'}).text
  
  url = soup.find('a', attrs={'class': 'btn-maroon'})['href']

  apply_link = base+url[1:]

  return summary, amount, str(criteria), awarding_body, deadline, apply_link

In [ ]:
for row in soup1.find('tbody').findAll('tr'):
  data = row.findAll('td')

  title = data[0].text.strip()
  categ = 'Merit Based' if data[2].text.strip() == 'No' else 'Need Based'
  sub_page_link = data[0].a['href']
  link = url + sub_page_link[1:]


  summary, amount, criteria, awarding_body, deadline, apply_link = sub_pages(url, link)

  ASU_scholarships = ASU_scholarships.append({'Name of Scholarship' : title, 'Description':summary, 'Awarding Body' : awarding_body, 'Type of Scholarship': 'Institutional',
                           'Category': categ, 'Application Link' : apply_link, 'Eligibility Criteria' : criteria, 'Application Dedline' : deadline, 'Award Amount' : amount}, ignore_index=True)



In [ ]:
ASU_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount
0,Alex K. Victory Memorial Scholarship,"In memory of Alex Victory, generous contribut...",asufoundationscholarships@asuep.org,Institutional,Merit Based,https://scholarships.asu.edu/,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'N...","Sunday, March 05, 2023","$1,500"
1,Betty Udell Marshall Memorial Scholarship,The Betty Udell Marshall Memorial Scholarship ...,asufoundationscholarships@asuep.org,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '2.75', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
2,Davidson Family Foundation Scholarship,Tom and Debbie Davidson have had a long-standi...,asufoundationscholarships@asuep.org,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
3,Geyser Family Scholarship,Jeff Geyser is proud of his long affiliation w...,asufoundationscholarships@asuep.org,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies


In [ ]:
body_list = np.asarray(['ASU Foundation', 'ASU Foundation in partnership with the donor', 'ASU Foundation in partnership with the donor', 'ASU Foundation in partnership with the donor.'])

In [ ]:
ASU_scholarships['Awarding Body'] = body_list

In [ ]:
ASU_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount
0,Alex K. Victory Memorial Scholarship,"In memory of Alex Victory, generous contribut...",ASU Foundation,Institutional,Merit Based,https://scholarships.asu.edu/,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'N...","Sunday, March 05, 2023","$1,500"
1,Betty Udell Marshall Memorial Scholarship,The Betty Udell Marshall Memorial Scholarship ...,ASU Foundation in partnership with the donor,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '2.75', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
2,Davidson Family Foundation Scholarship,Tom and Debbie Davidson have had a long-standi...,ASU Foundation in partnership with the donor,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
3,Geyser Family Scholarship,Jeff Geyser is proud of his long affiliation w...,ASU Foundation in partnership with the donor.,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies


In [ ]:
ASU_scholarships.to_csv('/content/drive/MyDrive/AlmaNotes/Top 5 US University Scholarships Scrapper/ASU_Scholarships.csv', index=False)

In [ ]:
ASU_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount
0,Alex K. Victory Memorial Scholarship,"In memory of Alex Victory, generous contribut...",ASU Foundation,Institutional,Merit Based,https://scholarships.asu.edu/,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'N...","Sunday, March 05, 2023","$1,500"
1,Betty Udell Marshall Memorial Scholarship,The Betty Udell Marshall Memorial Scholarship ...,ASU Foundation in partnership with the donor,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '2.75', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
2,Davidson Family Foundation Scholarship,Tom and Debbie Davidson have had a long-standi...,ASU Foundation in partnership with the donor,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies
3,Geyser Family Scholarship,Jeff Geyser is proud of his long affiliation w...,ASU Foundation in partnership with the donor.,Institutional,Need Based,https://scholarships.asu.edu/cas?destination=n...,"{'Minimum GPA:': '3.00', 'FAFSA Required:': 'Y...","Sunday, March 05, 2023",Varies


# MIT University

In [ ]:
MIT_scholarships = pd.DataFrame(columns=['Name of Scholarship', 'Description', 'Awarding Body', 'Type of Scholarship', 'Category', 'Application Link',
                                         'Eligibility Criteria', 'Application Dedline', 'Award Amount', 'Scholarship Benefits', 'No. of Scholarships',
                                         'Renewability', 'International Students Eligible'])

In [ ]:
MIT_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount,Scholarship Benefits,No. of Scholarships,Renewability,International Students Eligible


In [ ]:
MIT_url = 'https://collegedunia.com/usa/university/1087-massachusetts-institute-of-technology-cambridge/scholarship'

In [ ]:
MIT_html = requests.get(MIT_url)
soup2 = BeautifulSoup(MIT_html.text, 'lxml')

In [ ]:
MIT_html.status_code

200

In [ ]:
def mit_sub_pages(lnk):
  html = requests.get(lnk)
  soup = BeautifulSoup(html.text, 'lxml')

  summary = ''
  for line in soup.find('div', attrs={'class':'jsx-1117311724'}).findAll('p'):
    summary += line.text.strip()

  table = soup.findAll('table')[0]
  td_data = {'Scholarship Type': 0, 'Offered by': 0,'Organization':0, 'Application Deadline':0, 'No. of Scholarships': 0, 'Amount':0, 'Renewability': 0, 'International Students Eligible': 0, 'Scholarship Website Link' : 0}
  for rows in table.findAll('tr'):
    data = rows.findAll('td')
    td_data[data[0].text.strip()]  = data[1].text.strip()
  
  return summary, td_data['Scholarship Type'], td_data['Offered by'], td_data['Organization'], td_data['Application Deadline'], td_data['No. of Scholarships'], td_data['Amount'], td_data['Renewability'], td_data['International Students Eligible'], td_data['Scholarship Website Link']
    

In [ ]:
tables = soup2.findAll('table')
table2 = tables[1]

for rows in table2.findAll('tr')[1:]:
  data = rows.findAll('td')

  title = data[0].text.strip()
  benefits = data[1].text.strip()
  crit = data[2].text.strip()

  link = data[0].a['href']

  smry, s_type, offered_by, body, dedline, no_sholar, amount, renew, int_std_elg, apply_link =   mit_sub_pages(link)

  MIT_scholarships = MIT_scholarships.append({'Name of Scholarship' : title, 'Description' : smry, 'Awarding Body' : body, 'Type of Scholarship' : s_type, 'Category' : 'NA', 
                            'Application Link' : apply_link,'Eligibility Criteria' : crit, 'Application Dedline' : dedline, 'Award Amount': amount,
                            'Scholarship Benefits' : benefits, 'No. of Scholarships' : no_sholar, 'Renewability' : renew, 'International Students Eligible' : int_std_elg},
                           ignore_index = True)



In [ ]:
MIT_scholarships

,Name of Scholarship,Description,Awarding Body,Type of Scholarship,Category,Application Link,Eligibility Criteria,Application Dedline,Award Amount,Scholarship Benefits,No. of Scholarships,Renewability,International Students Eligible
0,Fulbright-Nehru Master’s Fellowships,The Fulbright-Nehru Master’s Fellowships are a...,United States- India Educational Foundation (U...,"Merit-Based, Need-Based",NA,https://www.usief.org.in/Fulbright-Nehru-Fello...,Minimum 55% in bachelor’s program;\r\n\t\t\tAt...,"May 17 ,2023",Variable,Tuition fees; partial living expenses; Healthc...,N/A,"Yes, Yearly",Yes
1,JN Tata Endowment for Higher Education of Indians,"JN Tata Endowment Scholarship, a one-time loan...",JN Tata Endowment,Merit-Based,NA,https://jntataendowment.org/,Over 60% in bachelor degree; Below 45 years of...,"Mar 07 ,2023",Variable,"Loan scholarship INR 1,00,000 and INR 10,00,00...",N/A,One Time Payment,Yes
2,Debesh-Kamal Scholarship for Higher Studies,Collegedunia TeamContent Curator | Updated o...,Ramakrishna Mission,"Merit-Based, Need-Based",NA,https://sriramakrishna.org,Below 30 years of age; Preferably beginning de...,N/A,"$1,385","INR 1,00,000 for tuition fees",N/A,One Time Payment,Yes
3,Narotam Sekhsaria Foundation Scholarships,"Veda BurmanUpdated on - Feb 23, 2023The Narota...",Narotam Sekhsaria Foundation,Merit-Based,NA,https://pg.nsfoundation.co.in/application-proc...,Over 60% in UG program; Below 30 years of age,"Mar 16 ,2023",Variable,Interest-free loan,N/A,One Time Payment,Yes
4,Inlaks Shivdasani Foundation Scholarships,The Inlaks Shivdasani Foundation Scholarships ...,Inlaks Shivdasani Foundation,Merit-Based,NA,https://www.inlaksfoundation.org/scholarships/...,Minimum 70% in bachelors program; Below 30 yea...,"Mar 30 ,2023","$100,000","Up to USD 100,000",N/A,One Time Payment,Yes
5,KC Mahindra Education Trust Scholarship,The KC Mahindra Scholarship and grants are pro...,K.C. Mahindra Education Trust,"Merit-Based, Need-Based",NA,https://www.kcmet.org/what-we-do-Scholarship-G...,At least 65% in UG program,"Mar 31 ,2023","$5,542","Interest free loan INR 4,00,000- INR 8,00,000",N/A,Yes,Yes


In [ ]:
MIT_scholarships['Category'] = np.asarray(['Government', 'Private', 'Private', 'Private', 'Private', 'Private'])

In [ ]:
MIT_scholarships.drop(['No. of Scholarships'], axis=1, inplace=True)

In [ ]:
MIT_scholarships.to_csv('/content/drive/MyDrive/AlmaNotes/Top 5 US University Scholarships Scrapper/MIT_Scholarships.csv', index=False)